In [ ]:
pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from transformers import pipeline
import torch
# device = torch.device("cpu")
from transformers import GPTJForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer,GPTJForCausalLM
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('bigscience/bloomz-560m')
model = AutoModelForCausalLM.from_pretrained('bigscience/bloomz-560m')

In [ ]:
df = pd.read_csv('covid19.csv')

In [ ]:
df.head()

In [ ]:
predictions = []
for index, row in df.head(500).iterrows():
    print(f"{index+1}")
    tweet = row['tweet']
    prompt = f"""
    You are an excellent data annotator.
    You know every incident and have common sense like humans.
    Your aim is to annotate the given text data to 2 labels namely rumor & non-rumor.
    Output should be strictly the label for the given data which is  either rumor or non-rumor.
    Here is the given text data: \n{tweet}.
    The label for this text is:
    """


    inputs = tokenizer(prompt, return_tensors="pt")
    #inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_length=512)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # print(response)

  # Extract the label from the model's response
    label = response.split(':')[-1].strip().lower()
    label_corrected = label.split("\n")[0]
    # Store the label directly in the DataFrame
    df.at[index, 'label'] = label_corrected
    print(label_corrected)

# Save the DataFrame to a CSV file
    df.to_csv('result-covid19.csv', index=False)
